# Monitoring setup for Bicycle Sharing Demand Prediction

This notebook shows how you can use the Evidently to:
* calculate prerformance and data drift for the model, performed as batch checks 
* log models quality & data drift using MLflow Tracking
* explore the result 

More examples are avaliable in the github: https://github.com/evidentlyai/evidently/tree/main/examples

Evidently docs: https://docs.evidentlyai.com/

Join our Discord: https://discord.com/invite/xZjKRaNp8b

In [ ]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

In [4]:
import datetime
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import json

from sklearn import datasets, ensemble, model_selection
from scipy.stats import anderson_ksamp

from evidently.metrics import RegressionQualityMetric, RegressionErrorPlot, RegressionErrorDistribution
from evidently.metric_preset import DataDriftPreset, RegressionPreset
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Bicycle Demand Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [12]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday']) 

In [13]:
raw_data.index = raw_data.apply(lambda row: datetime.datetime.combine(row.dteday.date(), datetime.time(row.hr)),
                                axis=1)

In [14]:
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
2011-01-01 00:00:00,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01 01:00:00,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01 02:00:00,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01 03:00:00,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01 04:00:00,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Model training 

In [15]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]#'weathersit']

In [16]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [18]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    reference[numerical_features + categorical_features],
    reference[target],
    test_size=0.3
)

In [19]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [20]:
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=0)

In [21]:
preds_train = regressor.predict(X_train)
preds_test = regressor.predict(X_test)

## Model validation

In [22]:
X_train['target'] = y_train
X_train['prediction'] = preds_train

X_test['target'] = y_test
X_test['prediction'] = preds_test

In [23]:
column_mapping = ColumnMapping()

column_mapping.target = 'target'
column_mapping.prediction = 'prediction'
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [24]:
regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=X_train.sort_index(), current_data=X_test.sort_index(),
                                  column_mapping=column_mapping)
regression_performance_report

/home/shoaib/mlflow/lib/python3.10/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/shoaib/mlflow/lib/python3.10/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/shoaib/mlflow/lib/python3.10/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


## Production model training

In [25]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

RandomForestRegressor(n_estimators=50, random_state=0)

In [26]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [27]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
reference['prediction'] = ref_prediction

In [28]:
regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=None, current_data=reference,
                                  column_mapping=column_mapping)
regression_performance_report

## Some time has passed - how is the model working?

In [29]:
current_prediction = regressor.predict(current[numerical_features + categorical_features])
current['prediction'] = current_prediction

### Week 1

In [31]:
regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
                                  column_mapping=column_mapping)
regression_performance_report

/home/shoaib/mlflow/lib/python3.10/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.



### Week 2

In [32]:
regression_performance_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])

regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'], 
                                            column_mapping=column_mapping)
regression_performance_report.show()

### Week 3

In [33]:
regression_performance_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])

regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'], 
                                            column_mapping=column_mapping)
regression_performance_report.show()

/home/shoaib/mlflow/lib/python3.10/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.



## Why the quality has dropped?

In [34]:
column_mapping_drift = ColumnMapping()

column_mapping_drift.target = target
column_mapping_drift.prediction = prediction
column_mapping_drift.numerical_features = numerical_features
column_mapping_drift.categorical_features = []

In [35]:
data_drift_report = Report(metrics=[
    DataDriftPreset(),
])

data_drift_report.run(
    reference_data=reference,
    current_data=current.loc['2011-02-14 00:00:00':'2011-02-21 23:00:00'],
    column_mapping=column_mapping_drift,
)
data_drift_report

## Let's customize the report!

In [36]:
from evidently.calculations.stattests import StatTest

def _anderson_stat_test(reference_data: pd.Series, current_data: pd.Series, feature_type: str, threshold: float):
    p_value = anderson_ksamp(np.array([reference_data, current_data]))[2]
    return p_value, p_value < threshold

anderson_stat_test = StatTest(
    name="anderson",
    display_name="Anderson test (p_value)",
    func=_anderson_stat_test,
    allowed_feature_types=["num"]
)

# options = DataDriftOptions(feature_stattest_func=anderson_stat_test, all_features_threshold=0.9, nbinsx=20)

In [37]:
the_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution(),
    DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
])


the_report.run(
    reference_data=reference,
    current_data=current.loc['2011-02-14 00:00:00':'2011-02-21 23:00:00'], 
    column_mapping=column_mapping_drift
)
the_report

/home/shoaib/mlflow/lib/python3.10/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/tmp/ipykernel_1671/3244802616.py:4: UserWarning:

p-value floored: true value smaller than 0.001

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which i

## Jupyter vs Automated Run?

To run this part of the tutorial you need to install MLflow (if not installed yet)

You can install MLflow with the following command: `pip install mlflow` or install MLflow with scikit-learn via `pip install mlflow[extras]`

More details:https://mlflow.org/docs/latest/tutorials-and-examples/tutorial.html#id5

In [ ]:
try:
    import mlflow
except:
    !pip install mlflow

In [1]:
import mlflow
#import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [2]:
experiment_batches = [
    ('2011-01-29 00:00:00','2011-02-07 23:00:00'),
    ('2011-02-07 00:00:00','2011-02-14 23:00:00'),
    ('2011-02-15 00:00:00','2011-02-21 23:00:00'),  
]

In [38]:
the_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution(),
    DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
])

the_report.run(
    reference_data=reference, 
    current_data=current.loc[experiment_batches[0][0]:experiment_batches[0][1]],
    column_mapping=column_mapping_drift
)

/home/shoaib/mlflow/lib/python3.10/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/tmp/ipykernel_1671/3244802616.py:4: UserWarning:

p-value capped: true value larger than 0.25

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/tmp/ipykernel_1671/3244802616.py:4: UserWarning:

p-value capped: true value larger than 0.25

/tmp/ipykernel_1671/324480261

In [39]:
logged_json = json.loads(the_report.json())

In [40]:
logged_json

{'version': '0.4.1',
 'metrics': [{'metric': 'RegressionQualityMetric',
   'result': {'columns': {'utility_columns': {'date': None,
      'id': None,
      'target': 'cnt',
      'prediction': 'prediction'},
     'num_feature_names': ['atemp',
      'hr',
      'hum',
      'mnth',
      'temp',
      'weekday',
      'windspeed'],
     'cat_feature_names': [],
     'text_feature_names': [],
     'datetime_feature_names': ['dteday'],
     'target_names': None},
    'current': {'r2_score': 0.8342907435814624,
     'rmse': 21.03964953999846,
     'mean_error': -6.26536170212766,
     'mean_abs_error': 13.52936170212766,
     'mean_abs_perc_error': 41.57714593080225,
     'abs_error_max': 94.34,
     'underperformance': {'majority': {'mean_error': -4.857251184834124,
       'std_error': 12.75642799788315},
      'underestimation': {'mean_error': -65.01666666666665,
       'std_error': 19.82673462929804},
      'overestimation': {'mean_error': 27.726666666666663,
       'std_error': 7.3900

In [41]:
[x['metric'] for x in logged_json['metrics']]

['RegressionQualityMetric',
 'RegressionErrorPlot',
 'RegressionErrorDistribution',
 'DatasetDriftMetric',
 'DataDriftTable']

In [42]:
logged_json['metrics'][0]['result']['current']['mean_error']

-6.26536170212766

In [43]:
logged_json['metrics'][3]['result']['drift_share']

0.5

In [44]:
#log into MLflow
client = MlflowClient()

#set experiment
mlflow.set_tracking_uri('http://localhost:5001/')
mlflow.set_experiment('Model Quality Evaluation')

#start new run
for date in experiment_batches:
    with mlflow.start_run() as run: #inside brackets run_name='test'
        
        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])

        # Get metrics
        the_report = Report(metrics=[
            RegressionQualityMetric(),
            RegressionErrorPlot(),
            RegressionErrorDistribution(),
            DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
        ])
        the_report.run(
            reference_data=reference, 
            current_data=current.loc[date[0]:date[1]],
            column_mapping=column_mapping_drift)
        logged_json = json.loads(the_report.json())
        
        me = logged_json['metrics'][0]['result']['current']['mean_error']
        mae = logged_json['metrics'][0]['result']['current']["mean_abs_error"]
        drift_share = logged_json['metrics'][3]['result']['drift_share']
        
        # Log metrics
        mlflow.log_metric('me', round(me, 3))
        mlflow.log_metric('mae', round(mae, 3))
        mlflow.log_metric('drift_share', round(drift_share, 3))

        print(run.info)

2023/08/22 17:10:27 INFO mlflow.tracking.fluent: Experiment with name 'Model Quality Evaluation' does not exist. Creating a new experiment.
/home/shoaib/mlflow/lib/python3.10/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/tmp/ipykernel_1671/3244802616.py:4: UserWarning:

p-value capped: true value larger than 0.25

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating

<RunInfo: artifact_uri='mlflow-artifacts:/6/4d8ede144dae4528a3c59b4b003d47b1/artifacts', end_time=None, experiment_id='6', lifecycle_stage='active', run_id='4d8ede144dae4528a3c59b4b003d47b1', run_name='capricious-crab-646', run_uuid='4d8ede144dae4528a3c59b4b003d47b1', start_time=1692706229438, status='RUNNING', user_id='shoaib'>


/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/tmp/ipykernel_1671/3244802616.py:4: UserWarning:

p-value capped: true value larger than 0.25

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the

<RunInfo: artifact_uri='mlflow-artifacts:/6/1ae50e349b424a32b30e4434cc7a26e5/artifacts', end_time=None, experiment_id='6', lifecycle_stage='active', run_id='1ae50e349b424a32b30e4434cc7a26e5', run_name='sassy-fowl-816', run_uuid='1ae50e349b424a32b30e4434cc7a26e5', start_time=1692706231127, status='RUNNING', user_id='shoaib'>


/home/shoaib/mlflow/lib/python3.10/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/tmp/ipykernel_1671/3244802616.py:4: UserWarning:

p-value floored: true value smaller than 0.001

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/tmp/ipykernel_1671/3244802616.py:4: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which i

<RunInfo: artifact_uri='mlflow-artifacts:/6/107801621228459e97d874c6dd41fb25/artifacts', end_time=None, experiment_id='6', lifecycle_stage='active', run_id='107801621228459e97d874c6dd41fb25', run_name='wistful-horse-233', run_uuid='107801621228459e97d874c6dd41fb25', start_time=1692706231955, status='RUNNING', user_id='shoaib'>


In [ ]:
#run MLflow UI (NOT recommented! It will be more convinient to run it directly from the TERMINAL)
#!mlflow ui

# Support Evidently
Enjoyed the tutorial? Star Evidently on GitHub to contribute back! This helps us continue creating free open-source tools for the community. https://github.com/evidentlyai/evidently